In [180]:
######### 始める前に設定必要の変数：###########

#### 1. BT : Academic申請後に生成される Bearer Token(BT)
#### 2. TWEET_LIMIT_you_set : 一回のクエリーの上限
#### 3. queryword_path : 辞書のエクセルファイル
#### 4. savedir_you_set : csvの保存フォルダー

BT = '######################'
TWEET_LIMIT_you_set = 1000                   ####  テストするときには1000とか設定した方がいいです
                                            #### 毎月の上限MAXは 10,000,000 なので　9,900,000にすることがおすすめです
queryword_path = './WordList.xlsx'          #### 辞書のエクセルファイル
savedir_you_set = './'

In [ ]:
import requests
import os
import json
import time
import pandas as pd


from datetime import datetime
from pytz import timezone




search_url = "https://api.twitter.com/2/tweets/search/all"




#与えられたパラメータからツイート取得条件(query_params)を生成する関数
def make_parm(keyword, start_time, end_time):

    #検索条件(今回はキーワード、開始時刻、修了時刻のみ)
    query_params = {'query': keyword ,
                    'tweet.fields': 'created_at',
                    'expansions': 'author_id,geo.place_id',
                    'start_time': start_time,
                    'end_time': end_time,
                    'user.fields': 'created_at,description,location,username,protected,verified',
                    'place.fields': 'contained_within,country,country_code,full_name,geo',
                    'max_results': 500,  #一回のqueryは５００で上限らしい
                    'next_token' : {} #次のページにいくためのparam
                   }
    return query_params


def create_headers(BT):
    headers = {"Authorization": "Bearer {}".format(BT)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        #raise Exception(response.status_code, response.text)
    
        print(response.status_code, response.text)
        
    return response   ######## to make use of  header information, return response instead of response.json()

def convert_timezone(normalized_data):
    normalized_data = normalized_data.copy().dropna(subset=['created_at'])
    normalized_data.created_at = pd.to_datetime(normalized_data.created_at, utc=True)
    normalized_data.index = pd.DatetimeIndex(normalized_data.created_at, name='created_at')
    normalized_data.index = normalized_data.index.tz_convert('Asia/Tokyo')
    
    
    # convert created_at_user column as well
    
    normalized_data.created_at_user = pd.to_datetime(normalized_data.created_at_user, utc=True)
    normalized_data.created_at_user = normalized_data.created_at_user.tz_convert('Asia/Tokyo')
    
    # sort dataframe by created time
    
    normalized_data.created_at = normalized_data.index
    normalized_data = normalized_data.sort_index()
    normalized_data = normalized_data.reset_index(drop=True)
    
    # rename index
    
    print('Timezone convertion completed')
    return normalized_data

def store_checkpoint(normalized_data, keyworddf):
    
    # store collected data to csv in case of error occurrence
    
    savedir = savedir_you_set
    
    kwindex = keyworddf['emotion']
        
    filename = kwindex + normalized_data.iloc[0]['created_at'].strftime('%Y%m%d%H%M') + '-' + \
               normalized_data.iloc[-1]['created_at'].strftime('%Y%m%d%H%M') + \
                '.csv'
        
    savepath = savedir + filename    
    
    normalized_data.to_csv(path_or_buf = savepath)
    print('Successfully saved at ' + savepath + ' at ' + \
              datetime.today().strftime("%Y/%m/%d-%H:%M"))
    

    
def json_to_df(json_response):
    
    normalized_data = pd.json_normalize(json_response['data'])
    normalized_users = pd.json_normalize(json_response['includes']['users']).set_index('id')
     
    normalized_users.rename({'id': 'author_id'}, inplace = True, axis = 1)
    normalized_data.rename({'geo.place_id': 'place_id'}, inplace = True, axis = 1)
            
    normalized = normalized_data.join(normalized_users, on = 'author_id', how = 'outer', rsuffix = '_user')
    
    if 'places' in json_response['includes'].keys():
        normalized_places = pd.json_normalize(json_response['includes']['places']).set_index('id')
        normalized_places.rename({'id': 'place_id'}, inplace = True, axis = 1)
        normalized = normalized.join(normalized_places, on = 'place_id', how = 'outer', rsuffix = '_place')
    
    normalized['text'] = normalized['text'].str.replace('\n', ' ')     #text内の\nを削除
    normalized['text'] = normalized['text'].str.replace('@', ' @')     #@前にスペースを追加
    
    return normalized
    


#実際にツイート取得をする関数(上の5つの関数を呼び出しながら)
def main(keyword, start_time, end_time):

    count = 0
    flag = True
    TWEET_LIMIT = TWEET_LIMIT_you_set
    
    
    normalized_data_old = pd.DataFrame(columns=['created_at','id', 'text', 'author_id',  
                                            'place_id', 'description',
                                            'name', 'verified', 'username', 'created_at_user', 'protected',
                                            'location', 'country_code', 'country', 'full_name', 'geo.type',
                                            'geo.bbox'])
    
    #cols = ['created_at', 'id', ' text']
    #normalized_data_old = pd.DataFrame(index=[], columns=cols)#create checkpoint dataframe
    
    query_params = make_parm(keyword['keyword'], start_time, end_time)#ここで取得条件が返される
    
    while flag:
        if count >= TWEET_LIMIT:
            break
    
        headers = create_headers(BT)
        time.sleep(1)
        
        response = connect_to_endpoint(search_url, headers, query_params)
        
        if response.status_code != 200:
            print(str(response.status_code) + 'Error occurred. Now trying to save csv')
            normalized_data_last = convert_timezone(normalized_data_old)
            store_checkpoint(normalized_data_last, keyword)
        
        json_response = response.json()  
        
        
        #print(json_response['data']) 
        #print(json_response) 
        
        normalized_data = json_to_df(json_response)
        
        normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
         
        normalized_data_old = normalized_data_new
        
        print("total:" + str(len(normalized_data_old)) + "tweets")
        if count % 5000 <= 1000:
            print('Finished collecting tweets at ' + str(normalized_data_old.iloc[-1]['created_at']))  
    

        result_count = json_response['meta']['result_count']
        
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            query_params['next_token'] = next_token
            count += result_count
            
            time.sleep(3)
            
            response = connect_to_endpoint(search_url, headers, query_params)
            time.sleep(3)
            
            
            if response.status_code == 200:
                
                continue
            
            elif response.status_code == 429:
                
                # store checkpoints when error occurs
                
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                # from https://github.com/mammalofski/Twitter-Scraper/blob/main/Scraper.py
                
                print('too many requests ... ')
                print('the header is ', response.headers)
                throttle_end_timestamp = int(response.headers.get('x-rate-limit-reset'))
                throttle_end_time = datetime.strftime(datetime.fromtimestamp(throttle_end_timestamp), "%H:%M:%S")
                time_to_wait = int(throttle_end_timestamp - datetime.now().timestamp()) + 10
                print('lets rest for', time_to_wait, 'seconds and wake up at', throttle_end_time)
                print('sleeping ...')
                time.sleep(time_to_wait)
                response = connect_to_endpoint(search_url, headers, query_params)
                
            else:
                normalized_data_last = convert_timezone(normalized_data_old)
                store_checkpoint(normalized_data_last, keyword)
                
                print('un expected error : ')
                print(response.status_code, response.text)
                print('sleep for 15 min')
                time.sleep( 15 * 60 + 10)  # rest 15min 
                print('restart at ' + datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"))
                
                response = connect_to_endpoint(search_url, headers, query_params)
             
            time.sleep(3)  # rate limit = 1 request/1 sec
            
            json_response = response.json()
            
            
            #append the current page of results to lists
            
            
            normalized_data = json_to_df(json_response)
        
            normalized_data_new = pd.concat([normalized_data_old, normalized_data], ignore_index=True)
            
            normalized_data_old = normalized_data_new
            
            print("total:" + str(len(normalized_data_old)) + "tweets")
            if count % 5000 <= 1000:
                print('Finished collecting tweets at ' + str(normalized_data_old.iloc[-1]['created_at']))  
    

        else:
            flag = False
            print('Last page of query results')
            
          
    
    # convert UTC +0 timezone to UTC +9 timezone
    normalized_data_last = convert_timezone(normalized_data_old)
    
#     if count % 500 <= 100:
#         print('Finished collecting tweets at ' + str(normalized_data_last.iloc[-1]['created_at']))
    
    
    # store checkpoint to csv
    
    store_checkpoint(normalized_data_last, keyword)
    
    #normalized_data_last.to_csv('../FullArchiveData_2021Apr/T20200331-0501.csv')#csvファイルに保存

In [181]:
# filter example #

# keyword + ' lang:ja -診断メーカー -募集中 -みんなからの匿名質問を募集中！ -DM -エロ -ホワイトライオン -日経新聞 -東京新聞 -朝日新聞デジタル -日本経済新聞 -毎日新聞 -産経新聞 -埼玉新聞 -@Sankei_news -青森ニュース -産経ニュース -#linenews -is:reply -is:retweet'

In [ ]:
filterrule='lang:ja -診断メーカー -募集中 -みんなからの匿名質問を募集中！ -DM -エロ -ホワイトライオン -日経新聞 -東京新聞 -朝日新聞デジタル -日本経済新聞 -毎日新聞 -産経新聞 -埼玉新聞 -@Sankei_news -青森ニュース -産経ニュース -#linenews -is:reply -is:retweet'


wordlist_df = pd.read_excel(queryword_path)
def generatequeryword(cate): #e.g. cate = 'T' for TENSION
    
    wordlist_cate = wordlist_df.dropna(subset = ['file name']) \
                   [wordlist_df['file name'].dropna().str.contains(cate)] \
                   ['orignal form'].to_list()
    query_str = '(' + ' OR '.join(wordlist_cate) + ')'
    query_str += ' '
    query_str += filterrule

    return query_str
    
newcatedict = {}

for cate in ['T','D','A','V','F','C']:
    newcatedict[cate] = generatequeryword(cate)
    
pd.DataFrame.from_dict(newcatedict, orient = 'index').to_csv('queryword.csv')

keywords_df = pd.read_csv('queryword.csv', names = ['emotion', 'keyword'], index_col = None, header = 0)


kw = keywords_df.iloc[3]
### keywords_df.iloc[0] : Tension
### keywords_df.iloc[1] : Depression
### keywords_df.iloc[2] : Anger
### keywords_df.iloc[3] : Vigor
### keywords_df.iloc[4] : Fatigue
### keywords_df.iloc[5] : Confusion


kw['keyword']  #### キーワードの確認

In [182]:
########## 2015-09-30T15:00:00Z : JPT 2015-10-01 00:00:00
########## 2020-10-31T15:00:00Z : JPT 2015-11-01 00:00:00

main(keyword = kw, start_time="2015-09-30T15:00:00Z", end_time="2020-10-31T15:00:00Z")

total:503tweets
total:1004tweets
total:1504tweets
Timezone convertion finished
Successfully saved at ./風しん201502201707-201504151611.csv at 2021/07/09-15:15


In [29]:
test = pd.read_csv('./###############.csv',usecols=['created_at','id', 'text', 'author_id'],dtype={'id':str})